In [ ]:
# Import packages
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pandas as pd

Using TensorFlow backend.


In [ ]:
# Read data and set sentiment based on "stars" & Tokenize data
df = pd.read_csv('../input/yelp_review.csv')
data = df[['text', 'stars']]
data['sentiment']=['pos' if (x>2) else 'neg' for x in data['stars']]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
print("done replacing")
data['text']= [x.encode('utf-8') for x in data['text']]
tokenizer = Tokenizer(nb_words = 2500, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [ ]:
# Create LSTM model and compile 
embed_dim = 128
lstm_out = 300
batch_size= 32
model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = X.shape[1], dropout=0.1))
model.add(LSTM(lstm_out, dropout_U = 0.1, dropout_W = 0.1))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size = 0.20, random_state = 36)
model.fit(X_train, Y_train, batch_size = batch_size, nb_epoch = 1,  verbose = 5)

In [ ]:
score, acc = model.evaluate(X_valid, Y_valid, verbose = 2, batch_size = batch_size)
print("Logloss : %.2f" % (score))
print("Validation set Accuracy: %.2f" % (acc))